# SBTi-Finance Tool - Target Reporting
This notebook is an example of how to use the tool to calculate your portfolio's temperature scores and to save results to be submitted to SBTi's Target Validation Team (TVT) for validation and approval of your own Science Based Target.

Please see the [methodology, guidance](https://sciencebasedtargets.org/wp-content/uploads/2020/07/Temperature-Scoring-Beta-Methodology.pdf) and the [technical documentation](http://getting-started.sbti-tool.org/) for more details on how to calculate portfolio temperature scores and how to submit targets to the SBTi for validation and approval. 

See 1_analysis_example for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular. 

## Setting up
This section imports SBTi tools, import data providers, and load the portfolio.

In [1]:
!pip install SBTi

Looking in indexes: http://rdnexus1.ortec.finance:8081/nexus/repository/pypi-all/simple
  Cloning git://github.com/OFBDABV/SBTi to c:\users\jorisc\appdata\local\temp\pip-req-build-aba5o4ib
  Created wheel for SBTi: filename=SBTi-0.5-py3-none-any.whl size=289058 sha256=ad07762bd53958360f024087ca8c7dffd1960f71e3a9999e9afd91757463afe0
  Stored in directory: C:\Users\jorisc\AppData\Local\Temp\pip-ephem-wheel-cache-h7w72d3x\wheels\9b\39\b9\ad6d643283a5dc18ada60798167ce0057729af6a4640ad4a16
Successfully built SBTi


In [3]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore
from SBTi.interfaces import ETimeFrames, EScope
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Download resources for Google Colab environment


In [4]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/example_portfolio.csv", "data/example_portfolio.csv")
if not os.path.isfile("utils.py"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/utils.py", "utils.py")

try:  # Import statement when run in Google Colabs
    from utils import  print_aggregations, print_grouped_scores, anonymize
except:  # Import statement when run locally
    from examples.utils import  print_aggregations, print_grouped_scores, anonymize

Specify the data files to use (portfolio and company data). In this example we are using dummy data. 

To change the files, first upload (when using Google Colab) or place the files to the data map (when running locally).
Next, change the file locations between the quotes below.

In [5]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")             # adjust the path according to your data
portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1") # adjust the path according to your data
companies = SBTi.utils.dataframe_to_portfolio(portfolio)
portfolio

,company_name,company_id,company_isin,weights,investment_value,engagement_target
0,"Advanced Micro Devices, Inc",US0079031078,US0079031078,6.36%,35000000,True
1,Adobe Systems Inc.,US00724F1012,US00724F1012,1.82%,10000000,False
2,Capgemini Group,FR0000125338,FR0000125338,1.82%,10000000,False
3,"Cisco Systems, Inc.",US17275R1023,US17275R1023,1.82%,10000000,False
4,Coca-Cola HBC AG,CH0198251305,CH0198251305,1.82%,10000000,False
5,CVS Health,US1266501006,US1266501006,1.82%,10000000,False
6,Danone,FR0000120644,FR0000120644,1.82%,10000000,False
7,Dell Technologies,US24703L1035,US24703L1035,1.82%,10000000,False
8,Delta Electronics,TW0002308004,TW0002308004,1.82%,10000000,False
9,L'Orï¿½al,FR0000120321,FR0000120321,1.82%,10000000,False


## Select calculation options
Configure computational settings for the temperature score. Below an example configuration with other options displayed in the comments

In [6]:
# Settings                                              # Other Options: 
time_frames = [SBTi.interfaces.ETimeFrames.MID]         # ETimeFrames: SHORT MID and LONG
scopes = [EScope.S1S2, EScope.S1S2S3]                   # EScopes: S1, S2, S3, S1S2 and S1S2S3
fallback_score = 3.2                                    # The options of the default score are 3.2, 3.9 or 4.5
aggregation_method = PortfolioAggregationMethod.WATS    # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
grouping = ['sector']                                   # Any column available on company level

In [7]:
temperature_score = TemperatureScore(
    time_frames=time_frames,
    scopes=scopes,
    fallback_score=fallback_score,
    aggregation_method=aggregation_method,
    grouping=grouping
)

## Calculate company and portfolio temperature scores

In [8]:
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)

## Checks
In this example, the following scores are calculated:
- Portfolio score
- Score for each sector
- Percentage covered by targets vs. default scores
- Percentage of SBTs in portfolio

In [9]:
# Portfolio score
print_aggregations(aggregated_scores)

Timeframe  Scope      Temp score
mid        S1S2       2.63
mid        S1S2S3     2.88


In [10]:
# Score per sector
print_grouped_scores(aggregated_scores)


                         Temp score
mid - S1S2
Communication Services   3.20
Consumer Discretionary   2.44
Consumer Staples         3.20
Energy                   3.20
Financials               3.20
Health Care              2.70
Industrials              2.36
Information Technology   1.32
Materials                3.20
Real Estate              3.28
Utilities                2.58

                         Temp score
mid - S1S2S3
Communication Services   3.20
Consumer Discretionary   3.08
Consumer Staples         3.20
Energy                   3.20
Financials               3.11
Health Care              3.05
Industrials              2.72
Information Technology   2.15
Materials                3.20
Real Estate              3.21
Utilities                2.73


In [11]:
# Percentage of score based on default score
amended_portfolio[['company_name', 'scope', 'company_id', 'temperature_results']]

,company_name,scope,company_id,temperature_results
0,"Advanced Micro Devices, Inc",S1S2,US0079031078,0.000000
2,"Advanced Micro Devices, Inc",S1S2S3,US0079031078,0.727326
3,Adobe Systems Inc.,S1S2,US00724F1012,0.000000
5,Adobe Systems Inc.,S1S2S3,US00724F1012,0.000000
6,Capgemini Group,S1S2,FR0000125338,1.000000
...,...,...,...,...
140,Company AD,S1S2S3,KE0000000030,1.000000
141,Company AE,S1S2,TH0000000031,1.000000
143,Company AE,S1S2S3,TH0000000031,1.000000
144,Company AF,S1S2,ID0000000032,1.000000


In [12]:
# Percentage of SBTs in portfolio
portfolio_coverage_tvp = PortfolioCoverageTVP()
coverage = portfolio_coverage_tvp.get_portfolio_coverage(amended_portfolio.copy(), PortfolioAggregationMethod.WATS)
print("Part of portfolio with SBTi approved targets is {c:.2f}%".format(c=coverage))

Part of portfolio with SBTi approved targets is 35.45%


## Save anonymized data for SBTi target validation
In order for the targets to be validated by SBTi, you can save your data locally. By running the anonymize function, you can replace company identifiers with meaningless substitutes.

In [13]:
portfolio, provider = anonymize(portfolio, provider)

In order to store the portfolio and provider data locally, two options apply:
1. You are running the SBTi tool locally or from Google Colab
2. You are running the SBTi tool from a Docker container

If you run the SBTi tool locally or from Google Colab, you:
- Specify and filenames in the cell below
- Run the cell below

In [14]:
portfolio_filename = 'portfolio.xlsx'
provider_filename = 'provider.xlsx'
portfolio.to_excel(portfolio_filename, index=False)

writer = pd.ExcelWriter(provider_filename, engine='openpyxl')
provider.data['fundamental_data'].to_excel(writer, sheet_name='fundamental_data')
provider.data['target_data'].to_excel(writer, sheet_name='target_data')
writer.save()

If you run the SBTi tool locally, you find the output files in the root folder of this notebook

If you run the SBTi tool from Google Colab, you:
- Click on the files icon in the left pane
- Click the three dots that appear after hovering over the file
- Download the file to your local machine